In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

import glob
import pandas as pd

import os, os.path

import time as tm

from datetime import datetime, date, time, timedelta
    
import shutil

import csv

In [2]:
#download csv file


def downloadCSV(FromDate, ToDate, TCKR, downloadDict):
   
    
    chrome_options = webdriver.ChromeOptions()
    prefs = {'download.default_directory' : downloadDict}
    chrome_options.add_experimental_option('prefs', prefs)

    path_to_chromedriver = r'C:\Users\Prasanth\Downloads\M.S. Data Science Academics\2nd Sem\Independent study\Data\NSEStocksData\chromedriver' 
    # change path as needed
    
    browser = webdriver.Chrome(executable_path = path_to_chromedriver,chrome_options=chrome_options)

    url = 'https://www.nseindia.com/products/content/equities/equities/eq_security.htm'

    browser.get(url)


    browser.find_element_by_id("symbol").clear()
    browser.find_element_by_id("symbol").send_keys(TCKR)
    tm.sleep(0.5)
    select = Select(browser.find_element_by_id('series'))
    select.select_by_value('EQ')
    tm.sleep(1.25)

    browser.find_element_by_id("rdDateToDate").click()
    tm.sleep(0.5)

    browser.find_element_by_id("fromDate").clear()
    browser.find_element_by_id("fromDate").send_keys(FromDate)
    tm.sleep(0.5)

    browser.find_element_by_id("toDate").clear()
    browser.find_element_by_id("toDate").send_keys(ToDate)
    tm.sleep(1)

    browser.find_element_by_id("get").click()
    
    
    try:
        
        element = WebDriverWait(browser, 600).until(
            EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, "Download file in csv"))
        )
        element.click()
        
    except:
        return None
    #browser.close()
    return None

In [3]:

def createAStock(companyTicker, companyListDate, tempDirFordownload):

    ticker = companyTicker
    
    startDate = companyListDate
    startDate_dt = datetime.strptime(startDate, '%d-%b-%y')
    startDate = startDate_dt.strftime('%d-%m-%Y')
    
    path =  tempDirFordownload
    
    startDate_dt = datetime.strptime(startDate, '%d-%m-%Y')
    
    endDate_dt = datetime.now()- timedelta(days=1)
    
    nextDate_dt = startDate_dt  #initializing 

    while nextDate_dt <= endDate_dt:
        fromDate = nextDate_dt
        toDate = min(endDate_dt, nextDate_dt + timedelta(days=365))
        nextDate_dt = nextDate_dt + timedelta(days=366)
        fromDate_str = fromDate.strftime('%d-%m-%Y')
        toDate_str = toDate.strftime('%d-%m-%Y')
        tm.sleep(1.5) 
        downloadCSV(fromDate_str, toDate_str, ticker, path)



    tm.sleep(15)  #waits for 45 seconds to finsh the download probably can be done better 

    #path = downloadDict  # use your path
    allFiles = glob.glob(path + "/*.csv")
    frame = pd.DataFrame()
    list_ = []
    for file_ in allFiles:
        df = pd.read_csv(file_,index_col=None, header=0)
        list_.append(df)
    frame = pd.concat(list_)
    frame['Date'] = pd.to_datetime(frame['Date'])
    frame.sort_values('Date',ascending=True, inplace=True)
    frame = frame.reset_index()
    frame = frame.drop('index', 1)
    frame.head()
    FinalLocation = r'C:\Users\Prasanth\Downloads\NSEStocksData\FinalStocks' + ticker + '.csv'
    frame.to_csv(FinalLocation, sep=',')
    
    dirForyearlyCSVs = r'C:\Users\Prasanth\Downloads\NSEStocksData\YearlyStocks' + ticker 
    
    if not os.path.exists(dirForyearlyCSVs):
        os.makedirs(dirForyearlyCSVs)

    filelist = glob.glob(os.path.join(path, "*.csv"))
    for f in filelist:
        shutil.copy(f,dirForyearlyCSVs)
        tm.sleep(0.25)
        os.remove(f)

    browserExe = "chrome"
    os.system("pkill "+browserExe)


In [4]:

path =  r'C:\Users\Prasanth\Downloads\NSEStocksData\DownloadCSVforStocks'

reader = csv.DictReader(open(r'C:\Users\Prasanth\Downloads\M.S. Data Science Academics\2nd Sem\Independent study\Data\NSEStocksData\Stocksfinal2colv3.csv'))

result = {}
for row in reader:
    result[row['SYMBOL']] = row['DATE']
    
for ticker, startDate in result.items():
    print( ticker, startDate)
    createAStock(ticker, startDate, path)


HCC 01-Apr-03
HCG 30-Mar-16
HCL-INSYS 01-Jan-97
HCLTECH 06-Jan-00
HDFC 23-Oct-96
HDFCBANK 01-Jan-96
HDFCLIFE 17-Nov-17
HDIL 24-Jul-07
HEG 01-Jan-96
HEIDELBERG 01-Jan-96
HERCULES 19-Feb-08
HERITGFOOD 27-Nov-96
HEROMOTOCO 11-Apr-03
HESTERBIO 13-Mar-15
HEXAWARE 14-Jun-02
HFCL 09-Mar-11
HGS 19-Jun-07
HIGHGROUND 24-Nov-16
HIKAL 14-Feb-01
HIL 01-Jan-96
HILTON 24-May-07
HIMATSEIDE 01-Jan-96
HINDALCO 08-Jan-97
HINDCOMPOS 08-May-96
HINDCOPPER 15-Sep-10
HINDNATGLS 15-Apr-09
HINDOILEXP 23-Sep-03
HINDPETRO 17-Jun-98
HINDSYNTEX 01-Jan-96
HINDUJAVEN 24-Apr-07
HINDUNILVR 01-Jan-96
HINDZINC 21-Nov-06
HIRECT 26-Dec-06
HITECHCORP 20-Dec-07
HITECHGEAR 21-Jan-02
HMVL 21-Jul-10
HONAUT 18-Jul-03


TimeoutException: Message: timeout
  (Session info: chrome=64.0.3282.140)
  (Driver info: chromedriver=2.35.528161 (5b82f2d2aae0ca24b877009200ced9065a772e73),platform=Windows NT 10.0.15063 x86_64)
